# Turing Machine Languages

## Introduction
This section explores the theoretical foundations of Turing machine languages, including recursively enumerable languages, and properties of these languages.

The following are example Python implementations of a Turing Machine, provided as starter code for use in other Python code examples.

In [1]:
# Simple representation of a Turing Machine configuration
class TuringMachine:
    def __init__(self, states, alphabet, tape_alphabet, transition_function, start_state, accept_state, reject_state):
        self.states = states
        self.alphabet = alphabet
        self.tape_alphabet = tape_alphabet
        self.transition_function = transition_function
        self.start_state = start_state
        self.accept_state = accept_state
        self.reject_state = reject_state
        self.tape = []
        self.head_position = 0
        self.current_state = start_state
    
    def step(self):
        if self.current_state in [self.accept_state, self.reject_state]:
            return False  # Machine has halted
        
        current_symbol = self.tape[self.head_position] if self.head_position < len(self.tape) else '△'
        if (self.current_state, current_symbol) in self.transition_function:
            new_state, write_symbol, direction = self.transition_function[(self.current_state, current_symbol)]
            
            # Write symbol
            if self.head_position < len(self.tape):
                self.tape[self.head_position] = write_symbol
            else:
                self.tape.append(write_symbol)
            
            # Move head
            if direction == 'R':
                self.head_position += 1
            elif direction == 'L':
                self.head_position = max(0, self.head_position - 1)
            
            self.current_state = new_state
            return True
        else:
            self.current_state = self.reject_state
            return False

    def run(self, input_string, max_steps=1000):
        """
        Run the Turing machine on the given input string.
        Returns:
        - 'accept' if the machine reaches the accept state
        - 'reject' if the machine reaches the reject state
        - 'loop' if the machine exceeds max_steps (likely in a loop)
        """
        # Initialize the tape with the input string
        self.tape = list(input_string)
        self.head_position = 0
        self.current_state = self.start_state
        
        steps = 0
        while steps < max_steps:
            if self.current_state == self.accept_state:
                return 'accept'
            elif self.current_state == self.reject_state:
                return 'reject'
            
            if not self.step():
                break
            steps += 1
        
        if steps >= max_steps:
            return 'loop'
        else:
            return 'reject'  # Halted without reaching accept state

            

## 1. Recursively Enumerable Languages

### 1.1 Defintion
A language $L$ is recursively enumerable if there exists a Turing machine $T$ such that:

* For every string $w \in L$, $T$ halts and accepts $w$
* For every string $s \notin L$, $T$ either rejects $s$ or loops forever

We say a recursively enumerable language $L$ is Turing-recognizable since there exists a Turing machine that accepts every string in $L$. In other words, the machine will eventually halt and accept any input that belongs to the language. Another way to describe this is that there’s a Turing machine that can enumerate all the strings in $L$, one by one. If a string is in $L$, the machine will eventually recognize it and accept it. Later in this section, we’ll introduce Turing-decidable languages, which are quite different from Turing-recognizable ones.

### 1.2 Examples

#### 1.2.1 Example 1
The following Turing machine has three states: 

* State 1 (START state)
* State 2 (intermediate state)
* State 3 (HALT state)

The transitions are:

* From state 1 to state 2: On reading 'a', write 'a', move Right (a, a, R)
* From state 2 to state 3 (HALT): On reading 'a', write 'a', move Right (a, a, R)
* From state 2 to state 1: On reading 'b', write 'b', move Right (b, b, R)
* From state 1 to state 1: On reading 'b', write 'b', move Right (b, b, R)
* From state 1 to state 1: On reading blank (Δ), write blank, move Right (Δ, Δ, R)

```{mermaid}
flowchart LR
    accTitle: A Example TM
    accDescr: a diagram representing a TM that has three states
    start((1 START))
    state2((2))
    halt[HALT 3]
    
    %% Self loop on start state
    start -->|"(b, b, R)/(Δ, Δ, R)"| start
    
    %% Transition from start to state2
    start -->|"(a, a, R)"| state2
    
    %% Transition from state2 to halt
    state2 -->|"(a, a, R)"| halt
    
    %% Transition from state2 back to start
    state2 -->|"(b, b, R)"| start
    
    %% Style to match the image
    classDef default fill:#fff,stroke:#333,stroke-width:1px
    classDef halt fill:#fff,stroke:#333,stroke-width:1px,rx:15,ry:15
    
    class halt halt
```

**Accept Class**: Based on the transitions, the machine accepts if and only if it reads $aa$ somewhere in the input string. More precisely, it accepts an input string if it finds an $a$ followed immediately by another $a$. Examples of strings in the accept class include $aab$, $baa$, $baaba$. 

* $ACCEPT(T)$: any strings with double $a$.

**Reject Class**: This machine doesn't have an explicit reject state, but it can implicitly reject if it reaches a configuration where no transition is defined. For a string ending in $a$ but without $aa$, the machine will process the string and never encounter two consecutive $a$'s. When it reaches the final $a$, it will transition from state 1 to state 2. Since there are no more characters, it will not move to state 3, but it also has no transition defined for state 2 with blank input. This causes the machine to reject the string (by halting without reaching the accept state). Examples of rejected strings include $ba$, $baba$, $bbba$.

* $REJECT(T)$: any strings without double $a$ and end in $a$.

**Loop Class**: By analyzing the two loop transitions at state 1, we see that the loop class contains all strings that do not contain the substring $aa$ and end in $b$. For such strings, the machine will process each symbol without ever encountering two consecutive $a$’s. When it reaches the final $b$,  it either remains in state 1 or transitions from state 2 back to state 1. With no further input to process, the machine enters an infinite loop in state 1, causing the machine to loop forever. Examples of strings causing loops include $b$, $ab$, $bab$.

* $LOOP(T)$: any strings without double $a$ and end in $b$.

Example Python Implementation:

In [2]:
# Sample Turing machine implementation
def create_aa_substring_tm():
    # Define the components
    states = {'1', '2', '3'}
    alphabet = {'a', 'b'}
    tape_alphabet = {'a', 'b', '△'}
    
    # Define the transition function based on the diagram
    transition_function = {
        # From state 1
        ('1', 'a'): ('2', 'a', 'R'),  # If in state 1 and read 'a', go to state 2
        ('1', 'b'): ('1', 'b', 'R'),  # If in state 1 and read 'b', stay in state 1
        ('1', '△'): ('1', '△', 'R'),  # If in state 1 and read blank, stay in state 1
        
        # From state 2
        ('2', 'a'): ('3', 'a', 'R'),  # If in state 2 and read 'a', go to state 3 (HALT/accept)
        ('2', 'b'): ('1', 'b', 'R'),  # If in state 2 and read 'b', go back to state 1
        # Note: No transition for (2, '△') - this will cause rejection if we encounter a blank in state 2
    }
    
    # Starting and accepting states
    start_state = '1'
    accept_state = '3'  # HALT state
    reject_state = 'reject'  # Not explicitly shown in the diagram, but needed for our implementation
    
    return TuringMachine(states, alphabet, tape_alphabet, transition_function, 
                         start_state, accept_state, reject_state)

# Function to test if a string contains 'aa'
def contains_aa(string):
    """
    Check if the string contains the substring 'aa'
    """
    return 'aa' in string

# Function to check if a string is in the reject class
def is_reject_class(string):
    """
    Check if the string is in the reject class:
    - Does not contain 'aa'
    - Ends with 'a'
    """
    return not contains_aa(string) and string.endswith('a')

# Function to check if a string is in the loop class
def is_loop_class(string):
    """
    Check if the string is in the loop class:
    - Does not contain 'aa'
    - Does not end with 'a'
    """
    return not contains_aa(string) and not string.endswith('a')

# Test the Turing machine
def test_tm():
    tm = create_aa_substring_tm()
    
    # Test cases for accept class
    accept_tests = ["aa", "aab", "baa", "baab", "ababaa"]
    print("ACCEPT CLASS TESTS (should all be 'accept'):")
    for test in accept_tests:
        result = tm.run(test)
        expected = "accept" if contains_aa(test) else "not accept"
        print(f"String: '{test}', Result: {result}, Expected: {expected}")
    
    # Test cases for reject class
    reject_tests = ["a", "ba", "baba", "bbba"]
    print("\nREJECT CLASS TESTS (should all be 'reject'):")
    for test in reject_tests:
        result = tm.run(test)
        expected = "reject" if is_reject_class(test) else "not reject"
        print(f"String: '{test}', Result: {result}, Expected: {expected}")
    
    # Test cases for loop class
    loop_tests = ["", "b", "bb", "bab"]
    print("\nLOOP CLASS TESTS (should all be 'loop'):")
    for test in loop_tests:
        result = tm.run(test)
        expected = "loop" if is_loop_class(test) else "not loop"
        print(f"String: '{test}', Result: {result}, Expected: {expected}")

# Run the tests
if __name__ == "__main__":
    test_tm()

ACCEPT CLASS TESTS (should all be 'accept'):
String: 'aa', Result: accept, Expected: accept
String: 'aab', Result: accept, Expected: accept
String: 'baa', Result: accept, Expected: accept
String: 'baab', Result: accept, Expected: accept
String: 'ababaa', Result: accept, Expected: accept

REJECT CLASS TESTS (should all be 'reject'):
String: 'a', Result: reject, Expected: reject
String: 'ba', Result: reject, Expected: reject
String: 'baba', Result: reject, Expected: reject
String: 'bbba', Result: reject, Expected: reject

LOOP CLASS TESTS (should all be 'loop'):
String: '', Result: loop, Expected: loop
String: 'b', Result: loop, Expected: loop
String: 'bb', Result: loop, Expected: loop
String: 'bab', Result: loop, Expected: loop


#### 1.2.2 Example 2
The following Turing machine has three states:

* START (1): The initial state
* 2: An intermediate state
* HALT: The accepting state

The transitions are:

* From START to 2:
    * When reading 'Δ' (blank), write 'b', move Right
    * When reading 'a', write 'b', move Right
* From START to HALT:
    * When reading 'b', write 'b', move Right
* From 2 to 2:
    * When reading 'a', write 'b', move Right
    * When reading 'b', write 'b', move Right
    * When reading 'Δ' (blank), write 'b', move Right

```{mermaid}
stateDiagram-v2
    accTitle: A Example TM
    accDescr: a diagram representing a TM that has three states
    START: START (1)
    INTERMEDIATE: 2
    HALT: HALT

    START --> INTERMEDIATE : (Δ,b,R)/(a,b,R)
    START --> HALT : (b,b,R)
    INTERMEDIATE --> INTERMEDIATE : (a,b,R)/(b,b,R)/(Δ,b,R)
```

This Turing machine accepts strings that start with $b$. Therefore, the language recognized by this machine is $L = b(a+b)^*$

**Accept Class**: Strings that start with $b$ are accepted. When the machine reads $b$ in the start state (1), it transitions directly to the HALT state. Examples: $b$, $ba$, $bb$, $baa$.

* $ACCEPT(T)$: L

**Reject Class**: This machine doesn't explicitly reject any strings - it either accepts or loops.

* $REJECT(T): \varnothing$

**Loop Class**: All strings that don't start with $b$ will cause the machine to loop. If the first symbol is $a$' or $Δ$, the machine transitions to state 2. Once in state 2, the machine loops indefinitely, as all transitions from 2 lead back to itself. Examples: $a$, $aa$, $ab$.

* $LOOP(T)$: $L'$, the complement of $L$.

Since we can construct a Turing machine that accepts strings in $L = b(a+b)^*$ and either rejects or loops on strings not in $L$, we conclude that $L$ is a recursively enumerable language.

Example Python Implementation:

In [3]:
# Create the Turing Machine accepting strings starting with 'b'
def create_start_with_b_tm():
    # Define states
    states = {'x1', 'x2', 'HALT'}
    
    # Define alphabet
    alphabet = {'a', 'b', '△'}
    tape_alphabet = {'a', 'b', '△'}
    
    # Define transition function according to the diagram
    transition_function = {
        # Transitions from x1 (START)
        ('x1', 'a'): ('x2', 'b', 'R'),  # (a,b,R) - read a, write b, move right, go to x2
        ('x1', 'b'): ('HALT', 'b', 'R'),  # (b,b,R) - read b, write b, move right, go to HALT
        ('x1', '△'): ('x2', 'b', 'R'),  # (△,b,R) - read blank, write b, move right, go to x2
        
        # Transitions from x2 (self-loop)
        ('x2', 'a'): ('x2', 'b', 'R'),  # (a,b,R) - read a, write b, move right, stay in x2
        ('x2', 'b'): ('x2', 'b', 'R'),  # (b,b,R) - read b, write b, move right, stay in x2
        ('x2', '△'): ('x2', 'b', 'R'),  # (△,b,R) - read blank, write b, move right, stay in x2
    }
    
    # Define start, accept, and reject states
    start_state = 'x1'
    accept_state = 'HALT'  # The diagram shows HALT as an accepting state
    reject_state = 'reject'  # Not shown in diagram but needed for implementation
    
    return TuringMachine(states, alphabet, tape_alphabet, transition_function, 
                         start_state, accept_state, reject_state)


# Function to analyze the language recognized by this TM
def analyze_start_with_b_tm():
    """
    Analyzes the language recognized by the Turing machine from the diagram.
    Returns a description of the language and examples.
    """
    tm = create_start_with_b_tm()
    
    # Test various inputs to understand the language
    test_strings = ['', 'a', 'b', 'aa', 'ab', 'bb', 'aaa', 'aab', 'aba', 'abb', 'baa', 'bab', 'bba', 'bbb']
    results = {}
    
    for string in test_strings:
        result = tm.run(string)
        results[string] = result
    
    # Based on the results, describe the language
    accepted_strings = [s for s, r in results.items() if r == 'accept']
    rejected_strings = [s for s, r in results.items() if r == 'reject']
    looping_strings = [s for s, r in results.items() if r == 'loop']
    
    print("Analysis of Turing Machine Language:")
    print(f"Accepted strings: {accepted_strings}")
    print(f"Rejected strings: {rejected_strings}")
    print(f"Looping strings: {looping_strings}")
    
    # Interpret the language pattern
    print("\nLanguage Description:")
    if 'b' in accepted_strings and not any(s for s in accepted_strings if not s.startswith('b')):
        print("This Turing machine accepts strings that start with 'b'")
    elif not accepted_strings:
        print("This Turing machine doesn't accept any strings (empty language)")
    else:
        print("This Turing machine accepts strings with pattern: [determine pattern from accepted strings]")
        
    return results


# Test the implementation
if __name__ == "__main__":
    # Create and test the Turing machine
    tm = create_start_with_b_tm()
    
    # Test a few specific strings
    test_cases = ['', 'a', 'b', 'aa', 'ab', 'ba', 'bb']
    
    print("Testing Turing Machine from the diagram:")
    for test in test_cases:
        result = tm.run(test)
        print(f"Input: '{test}', Result: {result}")
    
    # Analyze the language more broadly
    print("\n" + "="*50)
    analyze_start_with_b_tm()

Testing Turing Machine from the diagram:
Input: '', Result: loop
Input: 'a', Result: loop
Input: 'b', Result: accept
Input: 'aa', Result: loop
Input: 'ab', Result: loop
Input: 'ba', Result: accept
Input: 'bb', Result: accept

Analysis of Turing Machine Language:
Accepted strings: ['b', 'bb', 'baa', 'bab', 'bba', 'bbb']
Rejected strings: []
Looping strings: ['', 'a', 'aa', 'ab', 'aaa', 'aab', 'aba', 'abb']

Language Description:
This Turing machine accepts strings that start with 'b'


## 2. Recursive Language

### 2.1 Defintion
A language $L$ is recursive if there exists a Turing machine $T$ such that:

* For every string $w \in L$, $T$ halts and accepts $w$
* For every string $s \notin L$, $T$ rejects $s$
* $T$ must terminates on all inputs after a finite number of steps

Think of a recursive language as a language for which you can write a computer program that always finishes running and always tells you correctly whether the input belongs to the language. We say a recursive language $L$ is Turing-decidable since there exists a Turing machine that decides membership for every string in $L$. That is, the machine will halt on all inputs: it accepts strings that belong to $L$ and rejects those that don’t, with no possibility of looping forever. This makes recursive languages more powerful in a practical sense, because you can always determine whether a given string is in the language. Unlike Turing-recognizable languages, recursive languages guarantee both recognition and rejection through halting behavior. 

### 2.2 Examples
This is a simple Turing machine with just two states:

* START (the initial state)
* HALT (the accepting state)

There is a single transition:

* From START state to HALT state when reading 'a', writing 'a', and moving the tape head right (a,a,R)

```{mermaid}
stateDiagram-v2
    accTitle: A Simple TM
    accDescr: a diagram representing a TM that has two states
    START: START
    HALT: HALT

    START --> HALT : (a,a,R)
```

The language being accepted by this Turing machine contains all strings starting with $a$. 

* $ACCEPT(T): a(a+b)^*$
* $REJECT(T): \Lambda + b(a+b)^*$
* $LOOP(T): \varnothing$

This language is recursive (decidable) because the machine accepting it always terminates on any input. For any input string, the machine gives a definitive "yes" or "no" answer in a finite number of steps. 

### 2.3 Example Python Implementation

In [4]:
# Create the Turing machine accepting strings starting with 'a'
def create_starts_with_a_tm():
    # Define the components
    states = {'START', 'HALT'}
    alphabet = {'a', 'b', 'c'}  # You can add more symbols to the alphabet as needed
    tape_alphabet = {'a', 'b', 'c', '△'}  # Tape alphabet includes blank symbol
    
    # Define the transition function based on the diagram
    # Only one transition: from START to HALT when reading 'a'
    transition_function = {
        ('START', 'a'): ('HALT', 'a', 'R')
    }
    
    # Define the starting, accepting, and rejecting states
    start_state = 'START'
    accept_state = 'HALT'
    reject_state = 'REJECT'  # Not explicitly shown in the diagram, but needed for implementation
    
    return TuringMachine(states, alphabet, tape_alphabet, transition_function, 
                         start_state, accept_state, reject_state)

# Function to test if a string starts with 'a'
def starts_with_a(string):
    """
    Check if the string starts with the character 'a'
    """
    return string.startswith('a') if string else False

# Test the Turing machine
def test_tm():
    tm = create_starts_with_a_tm()
    
    # Test cases for strings that should be accepted
    accept_tests = ["a", "aa", "ab", "abc"]
    print("ACCEPT TESTS (strings starting with 'a'):")
    for test in accept_tests:
        result = tm.run(test)
        expected = "accept" if starts_with_a(test) else "reject"
        print(f"String: '{test}', Result: {result}, Expected: {expected}")
    
    # Test cases for strings that should be rejected
    reject_tests = ["", "b", "ba", "c", "cab"]
    print("\nREJECT TESTS (strings not starting with 'a'):")
    for test in reject_tests:
        result = tm.run(test)
        expected = "reject" if not starts_with_a(test) else "accept"
        print(f"String: '{test}', Result: {result}, Expected: {expected}")

# Run the tests
if __name__ == "__main__":
    test_tm()

ACCEPT TESTS (strings starting with 'a'):
String: 'a', Result: accept, Expected: accept
String: 'aa', Result: accept, Expected: accept
String: 'ab', Result: accept, Expected: accept
String: 'abc', Result: accept, Expected: accept

REJECT TESTS (strings not starting with 'a'):
String: '', Result: reject, Expected: reject
String: 'b', Result: reject, Expected: reject
String: 'ba', Result: reject, Expected: reject
String: 'c', Result: reject, Expected: reject
String: 'cab', Result: reject, Expected: reject


## 3. What Makes a Language Decidable or Recognizable?
By definition, if there exists at least one Turing machine that satisfies the criteria for recognizing or deciding a language, we classify the language accordingly:

* If there is a Turing machine that accepts all strings in the language and terminates on all inputs (accepts or rejects), then the language is called recursive (decidable).
* If there is a Turing machine that accepts all strings in the language but may loop forever on inputs not in the language, then the language is called recursively enumerable ( recognizable or semi-decidable).

The classification of the language depends on the existence of such a machine, not on the behavior of every possible machine for that language. So even if one Turing machine for a language loops on some inputs, that doesn’t mean the language is not recursive. There might be another Turing machine that terminates on all inputs. If even one such halting machine exists, the language is recursive. Conversely, if no machine can decide the language without possibly looping on some inputs, but there is at least one machine that can accept all valid strings, then the language is recursively enumerable but not recursive.

The key points are:

* Language classification is based on what is possible, not on what every implementation does.
* The presence of one looping machine does not mean the language is not recursive.
* What matters is whether a halting machine (for recursive) or an accepting machine (for recursively enumerable) exists.


## 4. Key Properties and Features

### 4.1 Dovetailing Construction

#### 4.1.1 What is Dovetailing
Dovetailing is a technique used to simulate multiple Turing machines at the same time, by interleaving their steps. Instead of running one machine to completion before starting the next, dovetailing runs a little bit of each machine in turn, switching back and forth rapidly. When we have multiple machines that might run forever, dovetailing ensures we don’t get stuck waiting on one machine that never halts. Instead, we give each machine a slice of time, so if any one of them halts, we will find out eventually.

#### 4.1.2 How does it work
Suppose we want to simulate two machines $T_1$ and $T_2$ on the same input $w$:

* Run step 1 of $T_1$
* Run step 1 of $T_2$
* Run step 2 of $T_1$
* Run step 2 of $T_2$
* Run step 3 of $T_1$
* Run step 3 of $T_2$
* … and so on.

You keep alternating steps between the two machines. As soon as either $T_1$ or $T_2$ halts and accepts, the dovetailing simulation can accept the input immediately.

### 4.2 Union of Recursively Enumerable Languages
The class of recursively enumerable languages is closed under union, which means if $L_1$ and $L_2$ are recursively enumerable languages, then their union $L_1 \cup L_2$ is also recursively enumerable.

To prove correctness, we construct a Turing machine that accepts $L_1 \cup L_2$ by simulating both machines for $L_1$ and $L_2$ in parallel. The new machine accepts any input string if either original machine accepts it, and it loops indefinitely on strings not in the union, as allowed for recursively enumerable languages. Let $T_1$ be a Turing machine that recognizes $L_1$. Let $T_2$ be a Turing machine that recognizes $L_2$. An important property of recursively enumerable languages is that Turing machines recognizing them are not required to terminate on strings not in the language. For strings $w$, the behavior of our union machine $T_3$ should be:

* If $w \in L_1$ or $w \in L_2$, then $T_3$ should accept $w$.
* If $w \notin L_1$ and $w \notin L_2$, then $T_3$ should not accept $w$, it can either:
    * Explicitly reject $w$, or
    * Loop forever (not terminate at all)

The key insight here is that for an recursively enumerable language recognizer, we only guarantee acceptance for strings in the language. For strings not in the language, there is no requirement to reach a specific decision. Our union machine $T_3$ should preserve this characteristic. This behavior is naturally achieved by the dovetailing construction: 

* If $w \in L_1$, then $T_1$ will eventually accept it, and $T_3$ will detect this and accept it.
* If $w \in L_2$, then $T_2$ will eventually accept it, and $T_3$ will detect this and accept it.
* If $w \notin L_1$ and $w \notin L_2$, then $T_3$ should either reject it or loop forever.

Now here’s an interesting question: can we build a Turing machine that accepts all strings in the union of two recursively enumerable languages, and rejects all strings not in the union without looping forever? If the answer is yes, meaning we can build such a machine or find an algorithm that always halts with the correct answer, then by definition, the union of the two languages would be recursive, not just recursively enumerable. The answer is actually No, we cannot always build a Turing machine that both accepts all strings in the union of two recursively enumerable languages AND rejects all strings not in the union, unless both original languages are already recursive (decidable). We’ll explore the detailed proof after we discuss another key property of recursive languages. Look at it from a different angle: the construction has to deal with situations where $T_1$ runs forever on some input $w \notin L_1$ and $T_2$ also runs forever on that same $w \notin L_2$. In these cases, there’s no way for the combined machine $T_3$ to decide in finite time that $w$ isn’t in the union. Since we are building an recursively enumerable recognizer, looping on strings not in the language is normal and actually expected. So let us revise the third step of the dovetailing construction as follows:

* If $w \notin L_1$ and $w \notin L_2$, then $T_3$ will run forever without terminating. 

In practical implementations of the union construction, we typically do not add explicit rejection logic. Instead:

* The dovetailing process continues indefinitely.
* If either $T_1$ or $T_2$ accepts, $T_3$ immediately accepts.
* If both $T_1$ and $T_2$ eventually reject, $T_3$ could be designed to run forever. Alternatively, we can modify $T_1$ and $T_2$ so that, instead of rejecting words they don’t accept, they run forever on those inputs.
* If either $T_1$ or $T_2$ runs forever, $T_3$ does't need any need special handling, because the original machine is already running indefinitely.

On the union machine $T_3$:

* $ACCEPT(T_3) = ACCEPT(T_1) + ACCEPT(T_2)$
* $REJECT(T_3) = \varnothing$
* $LOOP(T_3) = REJECT(T_1) + REJECT(T_2) + LOOP(T_1) + LOOP(T_2)$


### 4.3 Complement of Recursively Enumerable Languages
The complement of a language $L$, denoted as $L'$, is defined as the set of all strings over the alphabet that are not in $L$: $L' = \{w \in \Sigma^* | w \notin L\}$. The vertical bar $|$ means “such that.” If $L$ is a recursively enumerable language, its complement $L'$ is not necessarily recursively enumerable. Recursively enumerable languages are not closed under complement. We will prove this in the next section by introducing how Turing machines can be encoded, along with a few interesting new languages.


### 4.4 Complement of Recursive Languages
If the language $L$ is recursive, then its complement $L'$ is also recursive. In other words, the recursive languages are closed under complementation. To prove this, it is often easier to use Post machines instead of Turing machines. Although Post machines are not covered in detail in this textbook, they are explained in the further reading section. The key idea is to modify a Post machine for a recursive language so that it handles every possible input character by explicitly adding transitions to REJECT states. By adding REJECT states, we ensure the machine has a defined transition for every input symbol in every configuration. This turns the machine’s transition function into a total function. To recognize the complement of the language, we simply flip the ACCEPT and REJECT states. This creates a new Post machine that accepts exactly the strings not in the original language and still terminates on all inputs. Since this new machine also never loops, the complement language is recursive. Finally, because every Post machine has an equivalent Turing machine, this result holds for Turing machines as well.

However, this approach does not work for recursively enumerable languages. We cannot use the same reasoning to show that the complement of a recursively enumerable language is also recursively enumerable, because a Post machine for an recursively enumerable language might loop forever on some inputs. Simply swapping the ACCEPT and REJECT states doesn’t resolve this as any input that originally caused the machine to loop will still loop in the new machine. As a result, the machine cannot decide those inputs, and the complement language may not be recursively enumerable.

This property reveals several important insights:

* Decidability Means Complete Knowledge: If a problem is decidable (recursive), we have complete knowledge about both positive and negative instances. We can determine with certainty whether any string is in the language or not.
* Contrast with recursively enumerable Languages: recursively enumerable languages are not closed under complementation. There exist recursively enumerable languages whose complements are not recursively enumerable.


### 4.5 Recursively Enumerable Complement Implies Recursiveness
If a language $L$ is recursively enumerable and its complement $L'$ is also recursively enumerable, then $L$ is recursive (decidable). To prove this, we know that both languages are recursively enumerable, so there must exist two Turing machines $T_1$ and $T_2$ such that $T_1$ recognizes $L$ and $T_2$ recognizes $L'$:

* $ACCEPT(T_1) = L$, 
* $ACCEPT(T_2) = L'$

We can construct a Turing machine $T_3$ that decides $L$ as follows: On input $w$, run $T_1$ and $T_2$ in parallel using the dovetailing technique:

* If $T_1$ accepts $w$, then $T_3$ ACCEPT
* If $T_2$ accepts $w$, then $T_3$ REJECT

The reason why it works is because:

* If $w \in L$, then $T_1$ will eventually accept $w$, and $T_3$ will accept it
* If $w \notin L$, that means $w \in L'$, then $T_2$ will eventually accept $w$, and $T_3$ will reject it
* For any input $w$, either $T_1$ or $T_2$ must eventually accept it (as $w$ must be either in $L$ or in $L'$), so $T_3$ always terminates. Thus, $T_3$ decides $L$, making $L$ recursive.

When we say that we can build something, such as a machine or algorithm, it is important to back that up by showing how the construction is actually done. Demonstrating the “can” part is what turns a claim into a proven fact. However, in this case, the detailed construction process is quite lengthy and goes beyond the scope of this textbook. For those interested in a deeper understanding, the full construction is available in the further reading section.

In practical terms, this theorem says if you can enumerate all the "yes" instances AND all the "no" instances of a problem, then you can decide the problem. However, if you can only enumerate one set (either "yes" or "no" instances), then the problem might be undecidable.



### 4.6 Intersection of Recursively Enumerable Languages
If $L_1$ and $L_2$ are recursively enumerable languages, then their intersection $L_1 \cap L_2$ is also recursively enumerable. This means the class of recursively enumerable languages is closed under intersection operations. To prove this, we need to construct a Turing machine $L_3$ that recognizes $L_1 \cap L_2$. Unlike the union construction, we need both machines to accept for $L_3$ to accept:

* Step 1: Create a two-track Turing machine that copies the input from track 1 to track 2, then moves the head back to the start and begins running $L_1$ on track 1.
* Step 2: Modify $L_1$ to operate only on the top track, and change its HALT state so that it rewinds the tape head and then starts $L_2$.
* Step 3: Modify $L_2$ to operate only on the bottom track without changing its HALT state.

This combined machine first runs $L_1$ on the input. If $L_1$ accepts it, it then runs $L_2$ on the same input. The machine accepts only if both $L_1$ and $L_2$ accept. Therefore, it recognizes exactly the strings in the intersection of the two languages. Some inputs may still be rejected or cause one of the machines to loop forever. However, since we are only building a recognizer for a recursively enumerable intersection, this behavior is acceptable and does not affect the correctness of the construction.


## 5. Practice Exercises
### 5.1 Exercise 1:
Give an example of a language that is recursively enumerable but not recursive. Explain why it falls into this category.

### 5.2 Exercise 2:
Prove that every recursive language is recursively enumerable, but not every recursively enumerable language is recursive.

### 5.3 Exercise 3:
Let $L_1$ and $L_2$ be recursively enumerable languages. Which of the following must also be recursively enumerable? Justify your answer.

* $L_1 \cup L_2$
* $L_1 \cap L_2$
* $L_1 - L_2$


## 6. Further Reading
* "Introduction to the Theory of Computation" by Michael Sipser, Chapter 3
* "Introduction to Computer Theory" by Daniel I.A. Cohen, Chapter 23
* "Automata Theory, Languages, and Computation" by Hopcroft, Motwani, and Ullman, Chapter 8